In [1]:
import numpy as np
import pandas as pd

from scipy.optimize import minimize
from scipy.special import factorial

In [2]:
df = pd.read_csv("data/scores_and_odds.csv")

In [3]:
df

,date,home_team,away_team,home_score,away_score,tournament,home_odds,draw_odds,away_odds
0,2019-08-29,Israel,Italy,2.0,3.0,UEFA Euro qualification,3000.0,1600.0,-2000.0
1,2019-08-29,Lithuania,Croatia,1.0,2.0,UEFA Euro qualification,2150.0,900.0,-1000.0
2,2019-08-29,Faroe Islands,Wales,0.0,6.0,UEFA Euro qualification,3300.0,1200.0,-1429.0
3,2019-08-29,Denmark,Malta,8.0,0.0,UEFA Euro qualification,-10000.0,1825.0,4200.0
4,2019-08-29,Iceland,Hungary,4.0,1.0,UEFA Euro qualification,-714.0,760.0,1625.0
...,...,...,...,...,...,...,...,...,...
383,2022-06-27,Georgia,Republic of Ireland,0.0,9.0,FIFA World Cup qualification,8400.0,8900.0,-5000.0
384,2022-06-28,Netherlands,Belarus,3.0,0.0,FIFA World Cup qualification,-1000.0,4000.0,9900.0
385,2022-06-28,Estonia,Kazakhstan,4.0,2.0,FIFA World Cup qualification,275.0,240.0,153.0
386,2022-06-28,Moldova,Lithuania,1.0,1.0,FIFA World Cup qualification,530.0,310.0,-145.0


In [4]:
df["home_team"].unique().size

51

In [5]:
df["date"] = pd.to_datetime(df["date"])

df_data1 = pd.get_dummies(df.home_team, dtype=np.int64) - pd.get_dummies(df.away_team, dtype=np.int64)
df_data1["HFA"] = 1
df_data1["const"] = 1
df_data1["days"] = (pd.to_datetime("2022-07-06")-df.date).dt.days
df_data1["Goals"] = df["home_score"]

df_data2 = -df_data1.copy()
df_data2["HFA"] = 0
df_data2["const"] = 1
df_data2["days"] = (pd.to_datetime("2022-07-06")-df.date).dt.days
df_data2["Goals"] = df["away_score"]

df_data = pd.concat([df_data1, df_data2])

In [6]:
df_data

,Albania,Armenia,Austria,Azerbaijan,Belarus,Belgium,Bosnia-Herzegovina,Bulgaria,Croatia,Cyprus,...,Spain,Sweden,Switzerland,Turkey,Ukraine,Wales,HFA,const,days,Goals
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1042,2.0
1,0,0,0,0,0,0,0,0,-1,0,...,0,0,0,0,0,0,1,1,1042,1.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-1,1,1,1042,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1042,8.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1042,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,9,9.0
384,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,8,0.0
385,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,8,2.0
386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,8,1.0


In [7]:
with pd.option_context('display.max_columns', None):
    display(df_data.loc[380])

,Albania,Armenia,Austria,Azerbaijan,Belarus,Belgium,Bosnia-Herzegovina,Bulgaria,Croatia,Cyprus,Czech Republic,Denmark,England,Estonia,Faroe Islands,Finland,France,Georgia,Germany,Greece,Hungary,Iceland,Israel,Italy,Kazakhstan,Kosovo,Latvia,Lithuania,Luxembourg,Malta,Moldova,Montenegro,Netherlands,North Macedonia,Northern Ireland,Norway,Poland,Portugal,Republic of Ireland,Romania,Russia,Scotland,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,Wales,HFA,const,days,Goals
380,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,12,0.0
380,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,12,4.0


In [8]:
ind_cols = df_data.columns[1:-2]

In [9]:
ind_cols

Index(['Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium',
       'Bosnia-Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic',
       'Denmark', 'England', 'Estonia', 'Faroe Islands', 'Finland', 'France',
       'Georgia', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Israel', 'Italy',
       'Kazakhstan', 'Kosovo', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta',
       'Moldova', 'Montenegro', 'Netherlands', 'North Macedonia',
       'Northern Ireland', 'Norway', 'Poland', 'Portugal',
       'Republic of Ireland', 'Romania', 'Russia', 'Scotland', 'Serbia',
       'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey',
       'Ukraine', 'Wales', 'HFA', 'const'],
      dtype='object')

In [10]:
A = df_data[ind_cols].to_numpy()

In [11]:
X = np.ones_like(ind_cols, dtype=np.float64)

In [12]:
X

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

In [13]:
days = df_data["days"].to_numpy()
goals = df_data["Goals"].to_numpy()

In [14]:
h = 60

def log_likelihood(X):
    X1 = A.dot(X)
    return -((1/2)**(days/h)*(X1*goals - np.log(factorial(goals)) - np.exp(X1))).sum()

In [15]:
res = minimize(log_likelihood, X, method='BFGS', options={'disp': True})

Optimization terminated successfully.
         Current function value: 82.573450
         Iterations: 90
         Function evaluations: 6148
         Gradient evaluations: 116


In [16]:
ratings = pd.Series(res.x, index=ind_cols).sort_values(ascending=False)

In [17]:
ratings

England                1.818341
Spain                  1.805690
Belgium                1.564538
Sweden                 1.486001
Italy                  1.432286
Netherlands            1.398150
Iceland                1.363971
Scotland               1.218321
Norway                 1.215415
Austria                1.211231
Republic of Ireland    1.193398
Germany                1.159227
Switzerland            1.094898
France                 1.081583
Serbia                 0.895214
Poland                 0.879161
Denmark                0.749340
Romania                0.639400
Northern Ireland       0.637515
Russia                 0.572936
Finland                0.564222
Wales                  0.542317
Slovenia               0.509966
Slovakia               0.450723
Czech Republic         0.415839
Belarus                0.328000
Portugal               0.319649
HFA                    0.278229
Ukraine                0.180158
Hungary                0.143349
Greece                 0.045646
Croatia 

In [18]:
b = ratings["const"]
r1 = ratings["Netherlands"]
r2 = ratings["Denmark"]
hfa = ratings["HFA"]

In [19]:
lam_neth = np.exp(b+r1-r2+hfa)

In [20]:
lam_neth

2.446335638651978

In [21]:
lam_dk = np.exp(b+r2-r1)

In [22]:
lam_dk

0.5059809113680493

In [23]:
rng = np.random.default_rng(seed=0)

n = 10**7
scores = rng.poisson(lam=(lam_neth, lam_dk), size=(n,2))

In [24]:
scores[:3]

array([[2, 0],
       [7, 1],
       [3, 0]])

In [25]:
np.count_nonzero(scores[:,0] > scores[:,1])/n

0.8019058

In [26]:
np.count_nonzero(scores[:,0] < scores[:,1])/n

0.0581204

In [27]:
np.count_nonzero(scores[:,0] == scores[:,1])/n

0.1399738

In [28]:
ratings.to_csv("data/ratings.csv")